In [2]:
root = "E:/Personal Files/2022/data_mining/"
datasets =  ['bal','gla','hay','iri','new','win','zoo']

file_type = '.xls'
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import clone

def load_data(dataset):
    raw_data = pd.read_csv(root + dataset + file_type)
    label = raw_data[-1]
    data = raw_data.iloc[:,:-1]
    return data, label

def minmax(trainset):
    """归一化"""
    scaler = MinMaxScaler()
    trainset_new =  scaler.fit_transform(trainset)
    return trainset_new


# 每一折的指标存储
f1_save = []
accuracy_save = []

def cross_validation(dataset,label,model):
    """交叉验证"""
    X = dataset
    y = label
    # 每一折的指标存储
    f1_save = []
    accuracy_save = []
    skf = StratifiedKFold(n_splits=10, shuffle=True)
    # 10次10折交叉验证
    for i in range(10):
        for train_index, test_index in skf.split(X, y):
            # 复制一个纯净模型
            clf = clone(model)
            # K折
            X_train_folds = X[train_index]
            y_train_folds = y[train_index]
            X_test_fold = X[test_index]
            y_test_fold = y[test_index]
            # 对训练集做归一化
            X_train_folds = minmax(X_train_folds)
            # 训练模型
            clf.fit(X_train_folds, y_train_folds)
            # 预测测试集
            test_labels_pred = clf.predict(X_test_fold)
            # 计算指标
            f1_score_fold = f1_score(y_test_fold, test_labels_pred, average="weighted")
            acc_fold = accuracy_score(y_test_fold, test_labels_pred)
            #
            f1_save.append(f1_score_fold)
            accuracy_save.append(acc_fold)
            print(classification_report(y_test_fold, test_labels_pred))


    print('平均准确率为：')
    print(np.array(f1_save).mean())
    print('平均f1 score为：')
    print(np.array(accuracy_save).mean())


def get_accuracy():
    return accuracy_save

def get_f1():
    return f1_save

if __name__ == "__main__":

    for data in datasets:

